<a href="https://colab.research.google.com/github/bigfish951/Yolov7_2415485/blob/main/Colab_%EC%8B%9C%EC%9E%91%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 21 15:43:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
%%capture
!apt install git-lfs

In [5]:
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

Downloading:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/869M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset timit_asr downloaded and prepared to /root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/b11b576ddcccbcefa7c9f0c4e6c2a43756f3033adffe0fb686aa61043d0450ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [7]:
#file', 'audio', 'text'를 제외하고 제거
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))


show_random_elements(timit["train"].remove_columns(["audio", "file"]), num_examples=10)


,text
0,She had your dark suit in greasy wash water all year.
1,Do you have the yellow ointment ready?
2,The two artists exchanged autographs.
3,We'll serve rhubarb pie after Rachel's talk.
4,Most young rise early every morning.
5,Suburban housewives often suffer from the gab habit.
6,"Neither his appetites, his exacerbations, nor his despair were kin to yours."
7,I honor my mom.
8,The prowler wore a ski mask for disguise.
9,She had your dark suit in greasy wash water all year.


In [10]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'
#전사를 특수문자를 제거하고 소문자로 변환하는 함수
def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [11]:
timit = timit.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [12]:
#전처리된 전사 살펴보기
show_random_elements(timit["train"].remove_columns(["audio", "file"]))

,text
0,don't ask me to carry an oily rag like that
1,how do oysters make pearls
2,is a dream simply a mental or cerebral movie
3,her classical performance gained critical acclaim
4,be careful not to plow over the flower beds
5,his successors have adopted the opposite alternative
6,please shorten this skirt for joyce
7,his talk turns to what he calls the mess or sometimes this buzzing confusion
8,encyclopedias seldom present anecdotal evidence
9,she had your dark suit in greasy wash water all year


In [13]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [14]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'k': 0,
 'p': 1,
 'c': 2,
 'v': 3,
 'u': 4,
 'e': 5,
 ' ': 6,
 'g': 7,
 's': 8,
 'r': 9,
 'b': 10,
 'm': 11,
 'x': 12,
 'w': 13,
 'i': 14,
 'z': 15,
 'l': 16,
 'n': 17,
 't': 18,
 'o': 19,
 'h': 20,
 'y': 21,
 "'": 22,
 'q': 23,
 'j': 24,
 'd': 25,
 'f': 26,
 'a': 27}

In [16]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [17]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [22]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token = "[UNK]", pad_token = "[PAD]", word_delimiter_token = "|")

In [23]:
repo_name = "wav2vec2-base-timit-demo-colab"

In [24]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/bigfish951/wav2vec2-base-timit-demo-colab/commit/2e85c0ec29d35136f84d32cabd084962ae31f31e', commit_message='Upload tokenizer', commit_description='', oid='2e85c0ec29d35136f84d32cabd084962ae31f31e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bigfish951/wav2vec2-base-timit-demo-colab', endpoint='https://huggingface.co', repo_type='model', repo_id='bigfish951/wav2vec2-base-timit-demo-colab'), pr_revision=None, pr_num=None)